In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
input_path = Path('Х5_with_region_index_2024_population_patched_with_flags.xlsx')
df = pd.read_excel(input_path)

In [3]:
df

,new_id,Месяц,Трафик,Средний чек,"Дата открытия, категориальный","Торговая площадь, категориальный",Населенный пункт,Регион,Численность населения,Количество домохозяйств,...,"Трафик авто, в час","Маркетплейсы, доставки, постаматы (100 м)",Медицинские уч. и аптеки (300 м),Школы (300 м),Остановки (300 м),Продуктовые магазины (500 м),Пятерочки (500 м),Индекс_РИА_2024,traffic_flag,is_season
0,0,10,59662,976.170936,Средний по возрасту,Средний,Абинск г,Краснодарский край,38231,728,...,146.400000,0,0,0,0,0,0,76.58,1,0
1,0,5,56674,1025.462154,Средний по возрасту,Средний,Абинск г,Краснодарский край,38231,728,...,146.400000,0,0,0,0,0,0,76.58,1,0
2,0,1,51488,1158.150890,Средний по возрасту,Средний,Абинск г,Краснодарский край,38231,728,...,146.400000,0,0,0,0,0,0,76.58,1,0
3,3594,7,68039,1119.028697,Средний по возрасту,Средний,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,3827,768,...,406.000000,5,1,0,0,1,0,61.98,1,1
4,3594,6,64878,1112.584778,Средний по возрасту,Средний,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,3827,768,...,406.000000,5,1,0,0,1,0,61.98,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256706,21742,10,51676,1167.101083,Новый,Средний,Октябрьский рп,Волгоградская обл,6071,262,...,243.333333,1,1,0,0,1,0,53.66,1,0
256707,21742,11,51516,1252.914118,Новый,Средний,Октябрьский рп,Волгоградская обл,6071,262,...,243.333333,1,1,0,0,1,0,53.66,1,0
256708,21742,9,49593,1130.823998,Новый,Средний,Октябрьский рп,Волгоградская обл,6071,262,...,243.333333,1,1,0,0,1,0,53.66,1,0
256709,21742,12,52115,1461.929305,Новый,Средний,Октябрьский рп,Волгоградская обл,6071,262,...,243.333333,1,1,0,0,1,0,53.66,1,0


In [8]:
col_np = 'Населенный пункт'
col_region = 'Регион'
col_index = 'Индекс_РИА_2024'
col_is_season = 'is_season'
col_traffic_flag = 'traffic_flag'
col_pedestrian = 'Трафик пеший, в час'
col_check = 'Средний чек'
col_schools = 'Школы (300 м)'
col_size_cat = 'Торговая площадь, категориальный'
col_m = 'Месяц'

df[col_index] = pd.to_numeric(df[col_index], errors='coerce')
df[col_pedestrian] = pd.to_numeric(df[col_pedestrian], errors='coerce')
df[col_check] = pd.to_numeric(df[col_check], errors='coerce')
df[col_schools] = pd.to_numeric(df[col_schools], errors='coerce').fillna(0)

df['has_school'] = df[col_schools] > 0

size_map = {
    'Маленький': 'small',
    'Средний': 'medium',
    'Большой': 'large_plus',
    'Очень большой': 'large_plus',
}
df['size_group'] = df[col_size_cat].map(size_map)

# ищем колонку с численностью населения
pop_col = None
for c in df.columns:
    name = str(c)
    if 'числен' in name.lower() and 'насел' in name.lower():
        pop_col = c
        break
print('Колонка с населением:', pop_col)

if pop_col is not None:
    df[pop_col] = pd.to_numeric(df[pop_col], errors='coerce')
    df['population'] = df[pop_col]
else:
    df['population'] = np.nan

mask1 = (
    (df[col_is_season] == 1)
    & (df[col_traffic_flag] == 1)
    & (df[col_pedestrian] > 0)
    & df[col_index].notna()
    & df['size_group'].notna()
)
mask2 = (
    (df[col_is_season] == 0)
    & (df[col_traffic_flag] == 1)
    & (df[col_pedestrian] > 0)
    & df[col_index].notna()
    & df['size_group'].notna()
)


base_df_season = df[mask1].copy()
base_df_not_season = df[mask2].copy()
print('Размер base_df после фильтрации:', base_df_season.shape)
print('Размер base_df после фильтрации:', base_df_not_season.shape)
base_df_season[[col_m, col_np, col_region, col_pedestrian, col_check, col_index, 'population', 'has_school', 'size_group']].head()

Колонка с населением: Численность населения
Размер base_df после фильтрации: (60395, 24)
Размер base_df после фильтрации: (178677, 24)


,Месяц,Населенный пункт,Регион,"Трафик пеший, в час",Средний чек,Индекс_РИА_2024,population,has_school,size_group
3,7,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,89.857143,1119.028697,61.98,3827,False,medium
4,6,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,89.857143,1112.584778,61.98,3827,False,medium
10,8,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,89.857143,1275.068118,61.98,3827,False,medium
13,7,1-я Моква д,Курская обл,95.666667,683.123700,50.02,500,True,medium
15,8,1-я Моква д,Курская обл,95.666667,687.641711,50.02,500,True,medium


In [9]:
base_df_not_season[[col_m, col_np, col_region, col_pedestrian, col_check, col_index, 'population', 'has_school', 'size_group']].head()

,Месяц,Населенный пункт,Регион,"Трафик пеший, в час",Средний чек,Индекс_РИА_2024,population,has_school,size_group
0,10,Абинск г,Краснодарский край,68.25,976.170936,76.58,38231,False,medium
1,5,Абинск г,Краснодарский край,68.25,1025.462154,76.58,38231,False,medium
2,1,Абинск г,Краснодарский край,68.25,1158.150890,76.58,38231,False,medium
5,12,Абинск г,Краснодарский край,68.25,1031.000127,76.58,38231,False,medium
6,9,Абинск г,Краснодарский край,68.25,1035.065767,76.58,38231,False,medium
